In [2]:
import feedparser  
import pandas as pd 

RSS_URL = "https://rpp.pe/rss"

# Parsea (lee y procesa) el contenido del feed RSS
feed = feedparser.parse(RSS_URL)

# Prepara una lista vacía para almacenar las noticias que extraigamos
news_list = []

# Itera sobre las primeras 50 entradas (noticias) del feed
for entry in feed.entries[:50]:
    # Para cada noticia, crea un diccionario con los datos que nos interesan
    news_item = {
        'title': entry.get('title', 'Sin título'),           # Extrae el título, si no existe pone 'Sin título'
        'description': entry.get('summary', 'Sin descripción'), # Extrae el resumen, si no existe pone 'Sin descripción'
        'link': entry.get('link', '#'),                       # Extrae el enlace, si no existe pone '#'
        'published': entry.get('published', 'No disponible')  # Extrae la fecha de publicación
    }
    
    news_list.append(news_item) # Añade el diccionario de la noticia a nuestra lista


df_news = pd.DataFrame(news_list) # Convierte la lista de diccionarios en un DataFrame
df_news.head()

,title,description,link,published
0,Tacna: un sismo de magnitud 4.1 se sintió esta...,De acuerdo con información del Instituto Geofí...,https://rpp.pe/peru/tacna/tacna-un-sismo-de-ma...,"Sat, 18 Oct 2025 18:29:55 -0500"
1,Jesús María: paciente pide ayuda para que le a...,Rotafono de RPP | César Manuel Candiotti tiene...,https://rpp.pe/rotafono/servicios-publicos/jes...,"Sat, 18 Oct 2025 18:23:45 -0500"
2,"Temblor en Perú, hoy 18 de octubre: magnitud y...",Actualización EN VIVO del último sismo en Perú...,https://rpp.pe/lima/desastres-naturales/temblo...,"Thu, 16 Oct 2025 02:24:58 -0500"
3,Unión Comercio vs UCV Moquegua EN VIVO: ¿a qué...,Unión Comercio vs UCV Moquegua EN VIVO: sigue ...,https://rpp.pe/futbol/segunda-division/union-c...,"Sat, 18 Oct 2025 18:15:59 -0500"
4,Deportivo Garcilaso vs. Sporting Cristal: ¿a q...,Sporting Cristal va por un triunfo a la altura...,https://rpp.pe/futbol/descentralizado/sporting...,"Sat, 18 Oct 2025 18:15:44 -0500"


In [ ]:
import tiktoken 

sample_text = df_news['description'].iloc[0] # Selecciona la descripción de la primera noticia de nuestro DataFrame como muestra


encoding = tiktoken.get_encoding("cl100k_base") # Carga el codificador 


tokens = encoding.encode(sample_text) # Codifica el texto de muestra en tokens 
num_tokens = len(tokens)

print("--- Analizando el siguiente texto de muestra: ---") # Imprime el texto de muestra para tener contexto
print(f'"{sample_text}"')
print("-" * 50)


print(f" El texto de muestra tiene {num_tokens} tokens.") # El número de tokens calculado

# El modelo 'all-MiniLM-L6-v2' tiene un límite de 256 tokens.
# Comprobamos si nuestro texto excede este límite.
if num_tokens < 256:
    print(" Conclusión: El número de tokens está muy por debajo del límite del modelo (256).")
    print("No es necesario aplicar 'chunking' (división de texto) para las descripciones.")
else:
    print(" Conclusión: El texto podría ser demasiado largo. Se debería considerar aplicar 'chunking'.")

--- Analizando el siguiente texto de muestra: ---
"De acuerdo con información del Instituto Geofísico del Perú (IGP), el sismo se localizó a 14 kilómetros al sur del distrito de Ilabaya."
--------------------------------------------------
 El texto de muestra tiene 40 tokens.
✅ Conclusión: El número de tokens está muy por debajo del límite del modelo (256).
No es necesario aplicar 'chunking' (división de texto) para las descripciones.


In [ ]:
from sentence_transformers import SentenceTransformer 


model_name = "sentence-transformers/all-MiniLM-L6-v2" # Define el nombre del modelo que vamos a utilizar

model = SentenceTransformer(model_name) # Carga el modelo pre-entrenado.


descriptions = df_news['description'].tolist() # Extrae todas las descripciones de nuestro DataFrame a una lista 

# Usa el modelo para codificar todas las descripciones en vectores (embeddings)
# El parámetro show_progress_bar=True nos mostrará una barra de progreso útil
print(f"Generando embeddings para {len(descriptions)} noticias con el modelo '{model_name}'...")
embeddings = model.encode(descriptions, show_progress_bar=True)

# Imprime la "forma" (shape) del array de embeddings resultante
# Debería ser (50, 384), lo que significa 50 vectores, cada uno con 384 dimensiones.
print("\n--- Verificación de los Embeddings Generados ---")
print(f"Dimensiones del array de embeddings: {embeddings.shape}")
print(" Embeddings generados exitosamente.")

c:\Users\Usuario\anaconda3\envs\rpp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Usuario\anaconda3\envs\rpp\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Usuario\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In ord

Generando embeddings para 50 noticias con el modelo 'sentence-transformers/all-MiniLM-L6-v2'...


Batches: 100%|██████████| 2/2 [00:01<00:00,  1.98it/s]


--- Verificación de los Embeddings Generados ---
Dimensiones del array de embeddings: (50, 384)
✅ Embeddings generados exitosamente.


In [5]:
import chromadb 

client = chromadb.Client() # Crea un cliente de ChromaDB en memoria

# Define el nombre de nuestra colección de vectores
collection_name = "rpp_news"

# Obtiene o crea la colección.
collection = client.get_or_create_collection(name=collection_name)


# Prepara los metadatos.
metadatas = df_news[['title', 'link', 'published']].to_dict(orient='records')

# Prepara los IDs.
ids = [f"noticia_{i}" for i in range(len(df_news))]

# Añade los datos a la colección.
collection.add(
    embeddings=embeddings,
    documents=df_news['description'].tolist(),
    metadatas=metadatas,
    ids=ids
)

# Verifica cuántos ítems hay en la colección para confirmar que se guardaron los 50.
count = collection.count()

print("--- Almacenamiento en ChromaDB (en memoria) ---")
print(f" Se han añadido {count} documentos a la colección '{collection_name}'.")
print("La base de datos vectorial está lista para recibir consultas.")

--- Almacenamiento en ChromaDB (en memoria) ---
 Se han añadido 50 documentos a la colección 'rpp_news'.
La base de datos vectorial está lista para recibir consultas.


In [9]:
# Define la consulta o el tema que quieres buscar en las noticias
query_text = "Últimas noticias de futbol"
 
results = collection.query(
    query_texts=[query_text],  # Realiza la búsqueda en la colección de ChromaDB
    n_results=5  # le pide a la base de datos que nos devuelva los 5 resultados más similares.
)

retrieved_metadatas = results['metadatas'][0]
retrieved_documents = results['documents'][0]

# Creamos un DataFrame de pandas para mostrar los resultados de forma ordenada
df_results = pd.DataFrame({
    'title': [meta.get('title', 'N/A') for meta in retrieved_metadatas],
    'description': retrieved_documents,
    'link': [meta.get('link', '#') for meta in retrieved_metadatas],
    'date_published': [meta.get('published', 'N/A') for meta in retrieved_metadatas]
})

print(f"--- Mostrando los 5 resultados más relevantes para la consulta: '{query_text}' ---")

display(df_results)

--- Mostrando los 5 resultados más relevantes para la consulta: 'Últimas noticias de futbol' ---


,title,description,link,date_published
0,Unión Comercio vs UCV Moquegua EN VIVO: ¿a qué...,Unión Comercio vs UCV Moquegua EN VIVO: sigue ...,https://rpp.pe/futbol/segunda-division/union-c...,"Sat, 18 Oct 2025 18:15:59 -0500"
1,Comerciantes Unidos quiere seguir en Liga1: ve...,Comerciantes Unidos derrotó 1-0 a Alianza Atlé...,https://rpp.pe/futbol/descentralizado/comercia...,"Sat, 18 Oct 2025 17:15:33 -0500"
2,Deportivo Garcilaso vs. Sporting Cristal: ¿a q...,Sporting Cristal va por un triunfo a la altura...,https://rpp.pe/futbol/descentralizado/sporting...,"Sat, 18 Oct 2025 18:15:44 -0500"
3,Cienciano vs. Cusco FC EN VIVO vía L1 Max: jue...,Consulta todos los detalles de lo que será el ...,https://rpp.pe/futbol/descentralizado/ciencian...,"Sat, 18 Oct 2025 17:30:07 -0500"
4,Atlético Grau vs UTC EN VIVO: ¿a qué hora jueg...,Atlético Grau vs UTC EN VIVO: sigue la transmi...,https://rpp.pe/futbol/descentralizado/atletico...,"Sat, 18 Oct 2025 17:00:05 -0500"


In [11]:
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

# Le decimos a LangChain cómo generar embeddings usando el modelo que ya definimos.
embeddings_langchain = HuggingFaceEmbeddings(model_name=model_name)

#    Conectamos LangChain a nuestra base de datos ChromaDB que ya está en memoria.
#    Le pasamos el cliente, el nombre de la colección y la función de embeddings.
vectorstore = Chroma(
    client=client,
    collection_name=collection_name,
    embedding_function=embeddings_langchain
)

#    Un 'retriever' es el componente de LangChain especializado en hacer búsquedas.
#    Lo creamos a partir de nuestro vector store y le decimos que devuelva los 5 mejores resultados (k=5).
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})


# Define una nueva consulta para probar el retriever de LangChain
query_langchain = "agenda politica del día"

# Invoca al retriever con la consulta.
# Esto ejecutará todo el flujo de búsqueda de manera orquestada.
retrieved_docs = retriever.invoke(query_langchain)

print(f"--- Resultados de la búsqueda con LangChain para: '{query_langchain}' ---")

# Los resultados de LangChain vienen como una lista de objetos 'Document'.
# Iteramos sobre ellos para mostrar el contenido y los metadatos de forma clara.
for i, doc in enumerate(retrieved_docs):
    print(f"\n--- Resultado {i+1} ---")
    print(f"Descripción: {doc.page_content}")
    print(f"Título: {doc.metadata.get('title', 'N/A')}")
    print(f"Link: {doc.metadata.get('link', '#')}")
    print(f"Publicado: {doc.metadata.get('published', 'N/A')}")

--- Resultados de la búsqueda con LangChain para: 'agenda politica del día' ---

--- Resultado 1 ---
Descripción: A su llegada a Palacio, el presidente José Jerí le rindió homenaje al Cristo Moreno, que este sábado realiza su segundo recorrido por las calles del Cercado de Lima.
Título: José Jerí cargó el anda del Señor de los Milagros en la Plaza Mayor de Lima [VIDEO]
Link: https://rpp.pe/lima/actualidad/jose-jeri-cargo-el-anda-del-senor-de-los-milagros-en-la-plaza-mayor-de-lima-video-noticia-1659932
Publicado: Sat, 18 Oct 2025 13:00:54 -0500

--- Resultado 2 ---
Descripción: En la descripción del material, Tony Succar indicó que la colaboración está dedicada al Perú, en medio del clima de inseguridad y las manifestaciones por el descontento político.
Título: Isabela Merced y Tony Succar presentan versión en vivo del vals criollo 'Nada soy' [VIDEO]
Link: https://rpp.pe/famosos/celebridades/isabela-merced-y-tony-succar-presentan-version-en-vivo-del-vals-criollo-nada-soy-video-noticia-1